### Importação das bibliotecas

In [141]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import os

from datetime import datetime
from tqdm import tqdm

In [ ]:
jan = pd.read_csv(r"C:\Users\Yamac\OneDrive\Documentos\Programação\Trampo\Forecasting\Datasets\Sales_January_2019.csv")

In [65]:
jan['Quantity Ordered'][0]

'1'

### Leitura e tratamento dos dados

* Os dados estão separados por meses em planilhas diferentes
* Como as planilhas seguem a mesma estrutura, vou passar um loop e concatenar todos eles

In [68]:
folderpath = r"C:\Users\Yamac\OneDrive\Documentos\Programação\Trampo\Forecasting\Datasets"
datasets = []

for file in os.listdir(folderpath):
    dataset = pd.read_csv(os.path.join(folderpath, file))
    dataset = dataset[dataset['Order ID'] != 'Order ID']
    datasets.append(dataset)

In [156]:
df = pd.concat(datasets, axis=0, ignore_index=True)
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"


In [30]:
df.shape

(186850, 6)

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186495 entries, 0 to 186494
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          185950 non-null  object
 1   Product           185950 non-null  object
 2   Quantity Ordered  185950 non-null  object
 3   Price Each        185950 non-null  object
 4   Order Date        185950 non-null  object
 5   Purchase Address  185950 non-null  object
dtypes: object(6)
memory usage: 8.5+ MB


* Para identificar o mês a que as vendas se referem, é preciso alterar o formato da coluna "Order Date"
* Também vou separar a informação de horário em outra coluna

In [157]:
# Convert Order Data to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'])

In [158]:
# Drop null values
df.dropna(inplace=True)

In [159]:
df['Hour'] = df['Order Date'].dt.hour
# Remove hour from Ordeer Date
df['Order Date'] = df['Order Date'].dt.date
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Hour
0,176558,USB-C Charging Cable,2,11.95,2019-04-19,"917 1st St, Dallas, TX 75001",8
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07,"682 Chestnut St, Boston, MA 02215",22
3,176560,Google Phone,1,600,2019-04-12,"669 Spruce St, Los Angeles, CA 90001",14
4,176560,Wired Headphones,1,11.99,2019-04-12,"669 Spruce St, Los Angeles, CA 90001",14
5,176561,Wired Headphones,1,11.99,2019-04-30,"333 8th St, Los Angeles, CA 90001",9


In [160]:
# Sort dataframe cronologically
df.sort_values(by='Order Date', inplace=True)

In [161]:
df.rename(columns={'Quantity Ordered':'Amount', 'Price Each':'Price', 'Order Date':'Date', 'Purchase Address':'Address'}, inplace=True)

In [162]:
df['Hour'] = pd.to_datetime(df['Hour'], format='%H').dt.time
df['Date'] = pd.to_datetime(df['Date'])

In [164]:
# Create columns for city and state
df['City'] = df['Address'].apply(lambda x: x.split(',')[1])
df['State'] = df['Address'].apply(lambda x: x.split(',')[2].split(' ')[1])

#### Visualização dos dados

In [163]:
df.head()

,Order ID,Product,Amount,Price,Date,Address,Hour
68289,142066,27in 4K Gaming Monitor,1,389.99,2019-01-01,"110 Dogwood St, Seattle, WA 98101",22:00:00
76162,149579,Macbook Pro Laptop,1,1700,2019-01-01,"61 Lakeview St, Dallas, TX 75001",10:00:00
69491,143202,ThinkPad Laptop,1,999.99,2019-01-01,"129 Walnut St, Los Angeles, CA 90001",18:00:00
74481,147963,Flatscreen TV,1,300,2019-01-01,"655 Meadow St, Austin, TX 73301",11:00:00
73301,146844,Flatscreen TV,1,300,2019-01-01,"593 Church St, New York City, NY 10001",11:00:00


In [165]:
df['Amount'] = df['Amount'].astype('int')
df['Price'] = df['Price'].astype('float')

In [107]:
daily_sales = pd.DataFrame(df.groupby('Date')['Amount'].sum())
px.line(daily_sales, x=daily_sales.index, y='Amount', title='Daily Sales')

In [110]:
daily_revenue = pd.DataFrame(df.groupby('Date')['Price'].sum())
px.line(daily_revenue, x=daily_revenue.index, y='Price', title='Daily Revenue')

In [116]:
hyped_hours = pd.DataFrame(df.groupby('Hour')['Amount'].sum())
px.bar(hyped_hours, x=hyped_hours.index, y='Amount', title='Periods of the day with more sales')

In [118]:
# Number of unique products
df['Product'].unique()

array(['27in 4K Gaming Monitor', 'Macbook Pro Laptop', 'ThinkPad Laptop',
       'Flatscreen TV', 'USB-C Charging Cable', '34in Ultrawide Monitor',
       'Lightning Charging Cable', 'Bose SoundSport Headphones',
       'LG Dryer', 'AA Batteries (4-pack)', 'Apple Airpods Headphones',
       'Wired Headphones', 'AAA Batteries (4-pack)', '27in FHD Monitor',
       'iPhone', '20in Monitor', 'Google Phone', 'Vareebadd Phone',
       'LG Washing Machine'], dtype=object)

* É um pouco estranho, mas de fato, são apenas 19 produtos diferentes

In [134]:
product_sales_amount = pd.DataFrame(df.groupby('Product')['Amount'].sum()).sort_values('Amount', ascending=True)
px.bar(product_sales_amount, x='Amount', y=product_sales_amount.index, orientation='h', title='Amount sold by each product', text='Amount')

In [136]:
product_sales_rev = pd.DataFrame(df.groupby('Product')['Price'].sum()).sort_values('Price', ascending=True)
px.bar(product_sales_rev, x='Price', y=product_sales_rev.index, orientation='h', title='Revenue by each product', text='Price')	

In [137]:
sales_city = pd.DataFrame(df.groupby('City')[['Amount', 'Price']].sum()).sort_values('Amount', ascending=True)
px.bar(sales_city, x='Amount', y=sales_city.index, orientation='h', title='Amount sold by each city', text='Amount')

In [138]:
sales_state = pd.DataFrame(df.groupby('State')[['Amount', 'Price']].sum()).sort_values('Amount', ascending=True)
px.bar(sales_state, x='Amount', y=sales_state.index, orientation='h', title='Amount sold by each state', text='Amount')

##### Insights
* Na comparação entre quantidade vendida e receita, vemos que os produtos mais vendidos são os que geram a menor quantidade de receita
* É relevante ter noção da diferença do preço dos produtos, pois mais para a frente queremos calular uma função de erro baseada em prejuízo
* Existe uma clara preferência nos horários de compra, que são mais altos no horário de almoço e no começo da noite e mais baixos de madrugada, começo da manhã e meio da tarde, que são horários em que as pessoas estão dormindo ou trabalhando. Saber horários de maior consumo pode ajudar a direcionar melhor notificações e promoções

## Modelos de previsão

* Aqui começam os trabalhos de previsão das vendas
* A ideia é tentar prever como será a venda de cada produto nos próximos 30 dias

#### Modelos ingênuos

* Modelos ingênuos vão servir como formas simples de previsão
* A ideia é servir como benchmark para os modelos mais sofisticados

#### 1. Previsão baseada no dia anterior

In [139]:
df.head()

,Order ID,Product,Amount,Price,Date,Address,Hour,City,State
68289,142066,27in 4K Gaming Monitor,1,389.99,2019-01-01,"110 Dogwood St, Seattle, WA 98101",22:00:00,Seattle,WA
76162,149579,Macbook Pro Laptop,1,1700.00,2019-01-01,"61 Lakeview St, Dallas, TX 75001",10:00:00,Dallas,TX
69491,143202,ThinkPad Laptop,1,999.99,2019-01-01,"129 Walnut St, Los Angeles, CA 90001",18:00:00,Los Angeles,CA
74481,147963,Flatscreen TV,1,300.00,2019-01-01,"655 Meadow St, Austin, TX 73301",11:00:00,Austin,TX
73301,146844,Flatscreen TV,1,300.00,2019-01-01,"593 Church St, New York City, NY 10001",11:00:00,New York City,NY


In [ ]:
forecasts_last_day = []
eval_last_day = []

# We're using a loop to forecast each product individually
for product in df['Product'].unique():
    sales_filtered_last = df[df['Product'] == product].sort_values(by='Date', ascending=True)
    sales_filtered_last = sales_filtered_last[['Date', 'Amount']]
    sales_filtered_last = sales_filtered_last.groupby('Date').sum()
    sales_filtered_last.resample('D').ffill().fillna(0)  # Fill missing dates with 0

    train_last = sales_filtered_last.iloc[:-30] # Split in train and test by date -> in this case we want to use the last 30 days to test
    test_last = sales_filtered_last.iloc[-30:]
    
    train_last['Next day'] = train_last['Amount'].shift(1)  # Shift the amount of the previous day to the next day
    test_last['Next day'] = train_last['Next day'].iloc[-1]  # As the test won't have the real last day sales, we're filling it with the last 
                                                             # predicted value
    forecasts_last_day.append(train_last)
    forecasts_last_day.append(test_last)

    predicted = test_last['Next day'].values
    real = (test_last['Amount'] + 0.1).values  # Here we're adding 0.1 to avoid division by 0

    mape = np.mean(np.abs(predicted - real)/np.abs(real))
    mae = np.mean(np.abs(predicted - real))
    mpe = np.mean((predicted - real)/real)
    rmse = np.mean((predicted - real)**2)**.5    

    eval_last_day.append({'Product': product, 'MAPE':mape, 'MAE':mae, 'MPE':mpe, 'RMSE':rmse,
                        'Total Sold':real.sum(), 'Total Forecasted':predicted.sum()}) 

last_day = pd.concat(forecasts_last_day)
last_day['Next day'].fillna(0, inplace=True)
eval_last_df = pd.DataFrame(eval_last_day)